In [51]:
#neural net
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
from CNN_small_architecture import CNNSmall
import conv_homemade
import os

In [2]:
def tokenize(num):
    if num == 1:
        return torch.tensor(np.array([1., 0.]))
    else:
        return torch.tensor(np.array([0., 1.]))

MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

In [77]:
def transform_input(input_batch):
    return list(list(input_batch.detach().numpy()))

# compares two of the datatype transform_input produces
def compare(x, y):
    b_diff = []
    batch_size = len(x)

    for b in range(batch_size):
        c_diff = []
        channels = len(x[b])

        for c in range(channels):
            diff = np.sum(np.absolute(x[b][c] - y[b][c]))
            c_diff.append(diff)
        
        b_diff.append(c_diff)
    
    return b_diff

In [5]:
model_original = CNNSmall()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [74]:
input_original_test, label_test = next(iter(test_loader))
input_homemade_test = transform_input(input_batch=input_original_test)

In [50]:
weights = model_original.conv1.weight
biases = model_original.conv1.bias.detach().numpy()

out_c, in_c, r, c = weights.shape
conv1_filters = []

for f in range(out_c):
    filter_ = []
    kernels = []

    for kernel in list(weights[f,:,:,:]):
        kernels.append(kernel.detach().numpy())

    filter_.append(kernels)
    filter_.append(biases[f])
    conv1_filters.append(filter_)


In [60]:
conv1_homemade = conv_homemade.Conv(filters=conv1_filters, in_channels=in_c)

In [78]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# transformed tensor of original conv1 on test to datatype of homemade conv filter
out_original = transform_input(input_batch=model_original.conv1(input_original_test))

compare(out_homemade, out_original)

[[1.1187170039211186e-06, 1.5127357207156411e-06, 3.568385146715536e-06],
 [2.430800072740144e-06, 3.058141927331226e-06, 5.174706039362542e-06]]